In [ ]:
import numpy as np
import ipywidgets as ipw
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
output_notebook(hide_banner=True)

In [ ]:
# create an array of RGBA data
from bokeh.models import HoverTool
N = 20
img = np.empty((N, N), dtype=np.uint32)
view = img.view(dtype=np.uint8).reshape((N, N, 4))
for i in range(N):
    for j in range(N):
        view[i, j, 0] = int(255 * i / N)
        view[i, j, 1] = 158
        view[i, j, 2] = int(255 * j / N)
        view[i, j, 3] = 255

p = figure(plot_width=400, plot_height=400, x_range=(0, 10), y_range=(0, 10),
#           tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")]
          )

hv = HoverTool(
    tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])

im = p.image_rgba(image=[img], x=[0], y=[0], dw=[10], dh=[10])
p.add_tools(hv)

h = show(p, notebook_handle=True)


In [ ]:
view.shape

In [ ]:
from bokeh.models import LassoSelectTool
p.add_tools(LassoSelectTool())
push_notebook(handle=h)

In [ ]:
def rgba_to_ints(rgba):
    a = (rgba * 255).astype(int)
    return ((a[..., 3] << 24) + (a[..., 2] << 16) + (a[..., 1] << 8) + (a[..., 0]))

In [ ]:
# from bokeh.plotting import figure, output_file, show
# from bokeh.models import ColorBar, ColorMapper

N = 5000
x = np.linspace(0, 10, N)
y = np.linspace(0, 10, N)
xx, yy = np.meshgrid(x, y)
d = np.sin(xx)*np.cos(yy) + np.random.random([N, N])
# d = np.random.random([N, N, 50])

p = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")],
#           y_axis_type="log"
          )
p.x_range.range_padding = p.y_range.range_padding = 0




# color_mapper = ColorMapper(palette="Viridis256")

from matplotlib import cm
cmap = cm.get_cmap('viridis')

# must give a vector of image data for image parameter

# a = d[..., 0].copy()
# a[:50, :50] = np.NaN # float('nan')

# r2 = p.image(image=[d], x=1.3, y=1.7, dw=102, dh=200, palette='Greys256',
#              level="image")


r = p.image(image=[d], x=1.3, y=1.7, dw=102, dh=200,
            level="image",
           palette='Viridis256')
# dmasked = d[..., 0].copy()
# dmasked[:50, :50] = -100

p.grid.grid_line_width = 0.5

# color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12)

# p.add_layout(color_bar, 'right')

r.glyph.color_mapper.nan_color = (0, 0, 0, 0)

# output_file("image.html", title="image.py example")
# with out:
h = show(p, notebook_handle=True)
# h2 = show(p, notebook_handle=True)


In [ ]:
import ipywidgets as ipw
def update(change):
#     print(r.data_source.data['image'])
#     r.data_source.data['image'][0] = d[..., change["new"]]
    r.data_source.data = {'image': [d[..., change["new"]]]}
    push_notebook(handle=h)
sl = ipw.IntSlider(max=49)
sl.observe(update, names="value")
sl

In [ ]:
N = 1000
M = 500
xx = np.linspace(0.0, 12.0, N)
yy = np.linspace(0.0, 6.0, N)
x, y = np.meshgrid(xx, yy)
a = [np.sin(np.sqrt(x**2 + y**2))]
a.append(np.random.normal(a[0] * 0.1, 0.05))


p = figure(title="simple line example", plot_height=300, plot_width=600, y_range=(-5,5),
           background_fill_color='#efefef')
r1 = p.line(xx, a[0][:, 0], color="#8888cc", line_width=1.5, alpha=0.8)
r2 = p.line(xx, a[1][:, 0], color="#8888cc", line_width=1.5, alpha=0.8)
h = show(p, notebook_handle=True)

In [ ]:
r1.data_source.data['x'] = np.arange(10.)
r1.data_source.data['y'] = np.arange(10.)
push_notebook(handle=h)

In [ ]:
r1.data_source.data['y']

In [ ]:
sl = ipw.IntSlider(
                value=0,
                min=0,
                max=M,
                step=1,
                continuous_update=True,
                readout=True)

def update_y(change):
    r1.data_source.data['y'] = a[0][:, change["new"]]
    r2.data_source.data['y'] = a[1][:, change["new"]]
    push_notebook(handle=h)
#     for i, line in enumerate(ax.lines):
#         line.set_ydata(a[i][:, change["new"]])
sl.observe(update_y, names="value")
sl

In [ ]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import Figure, output_file
from bokeh.io import push_notebook, show, output_notebook

output_notebook()

# output_file("js_on_change.html")

x = [x*0.005 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(x=x, y=y))

plot = Figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var f = cb_obj.value
    var x = data['x']
    var y = data['y']
    for (var i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    source.change.emit();
""")

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
slider.js_on_change('value', callback)

layout = column(slider, plot)

show(layout)

In [ ]:
# from bokeh.models import (BasicTicker, ColorBar, ColumnDataSource,
#                           LinearColorMapper, PrintfTickFormatter,)
# from bokeh.transform import transform

# colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
# mapper = LinearColorMapper(palette=colors, low=0.0, high=1.0)




# p = figure()
# N = 2000
# M = 1000
# x = np.arange(float(N))
# y = np.arange(float(M))
# xx, yy = np.meshgrid(x, y)
# z = np.random.random([M, N]).flatten()

# data = {'horizontal': xx.flatten(),
#         'vertical': yy.flatten(),
#         'z': z}

# source = ColumnDataSource(data=data)

# p.rect(x="horizontal", y="vertical", width=1, height=1, source=source,
#        line_color=None, fill_color=transform('z', mapper))

# # p.rect(x=xx.flatten(), y=yy.flatten(), height=1,
# #        line_color=None, fill_color=transform(z, mapper))
# show(p)

In [ ]:
from bokeh.io import show
from bokeh.models import Button, CustomJS

def do_hello(c = None):
    print("hello")
    print(c)

button = Button(label="Foo", button_type="success")
# button.js_on_click(CustomJS(code="console.log('button: click!', this.toString())"))
# button.on_click(do_hello)
# button.js_on_click(CustomJS.from_py_func(do_hello))



def show_app(layout):

    from bokeh.application import Application

    from bokeh.application.handlers import FunctionHandler
    from bokeh.io import show

    def modify_doc(doc):
        doc.add_root(layout)

    handler = FunctionHandler(modify_doc)
    app = Application(handler)

    return show(app)

button.on_click(do_hello)

show_app(button)

In [ ]:
import numpy as np

from bokeh.models import ColorBar, LogColorMapper
from bokeh.plotting import figure, output_file, show

output_file('color_bar.html')

def normal2d(X, Y, sigx=1.0, sigy=1.0, mux=0.0, muy=0.0):
    z = (X-mux)**2 / sigx**2 + (Y-muy)**2 / sigy**2
    return np.exp(-z/2) / (2 * np.pi * sigx * sigy)

X, Y = np.mgrid[-3:3:100j, -2:2:100j]
Z = normal2d(X, Y, 0.1, 0.2, 1.0, 1.0) + 0.1*normal2d(X, Y, 1.0, 1.0)
image = Z * 1e6

color_mapper = LogColorMapper(palette="Viridis256", low=1, high=1e7)

plot = figure(x_range=(0,1), y_range=(0,1), toolbar_location=None)
plot.image(image=[image], color_mapper=color_mapper,
           dh=[1.0], dw=[1.0], x=[0], y=[0])

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12)

plot.add_layout(color_bar, 'right')

show(plot)

In [ ]:
from bokeh.layouts import row
N = 300
x = np.linspace(0, 10, N)
y = np.linspace(0, 10, N)
xx, yy = np.meshgrid(x, y)
# d = np.sin(xx)*np.cos(yy) + np.random.random([N, N])
d = np.random.random([N, N, 10])

p = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")],
           plot_width=500, plot_height=500)
p.x_range.range_padding = p.y_range.range_padding = 0

# must give a vector of image data for image parameter
r = p.image(image=[d[..., 0]], x=1.3, y=1.7, dw=102, dh=200, palette="Viridis256", level="image")
p.grid.grid_line_width = 0.5
p.toolbar.logo = None



p2 = figure(plot_width=60, plot_height=500)
# p2 = figure()
p2.x_range.range_padding = p2.y_range.range_padding = 0

# must give a vector of image data for image parameter
r2 = p2.image(image=[np.arange(128).reshape(128, 1)], x=0, y=0, dw=1.0, dh=1.0, palette="Viridis256", level="image")
# p2.grid.grid_line_width = 0.0
p2.xgrid.visible = False
p2.ygrid.visible = False
p2.toolbar.logo = None
p2.toolbar_location = None
p2.xaxis.major_label_text_font_size = '0pt'
p2.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
p2.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks

# output_file("image_with_cbar.html")

h = show(row(p, p2), notebook_handle=True)
# show(row(p, p2))
# show(p)


In [ ]:
from bokeh.io import export_svg
export_svg(p, filename="plot.svg")